In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock

In [2]:
m = wt.watertap_setup(dynamic = False)

In [3]:
wt.case_study_trains.train = {"case_study": "Carlsbad",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

In [4]:
# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

wt.case_study_trains.source_water = {"case_study": "Carlsbad", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Seawater"}

In [5]:
m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.

Carlsbad
sw_onshore_intake
coag_and_floc
tri_media_filtration
sulfuric_acid_addition
sodium_bisulfite_addition
cartridge_filtration
ro_deep
ro_deep_scnd_pass
lime_softening
co2_addition
chlorination
ammonia_addition
treated_storage
backwash_solids_handling
surface_discharge
municipal_drinking
landfill


In [6]:
wt.display.show_train2(model_name=m)

In [7]:
wt.run_water_tap(m = m, solver_results = False, print_model_results = False)

degrees_of_freedom: 0


In [8]:
def get_results_table():

    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']
    big_dict = {}
    # for letter in ['A', 'B', 'C']:
    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            #print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            for py, ex in dict(zip(py_var, excel_var)).items():
                #print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    #print(f'Python --> {num}')
                except ZeroDivisionError:
                    #print(f'Python --> {0} - ERROR')
                    py_vals.append(0)
                py_dict = dict(zip(py_var, py_vals))
                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.max()
                #print(f'Excel --> {num}\n\n')
                excel_vals.append(num)
                excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict
    else:
        pass 



    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                           orient='index')
    
    return df

In [9]:
get_results_table().head()

fixed_cap_inv  total_cap_investment  \
sw_onshore_intake    python      20.980306             22.060791   
                     excel       20.982722             22.063332   
coag_and_floc        python      11.765912             12.371856   
                     excel       16.110699             16.940400   
tri_media_filtration python      12.751314             13.408007   

                             cat_and_chem_cost  electricity_cost  \
sw_onshore_intake    python           0.000000          1.984638   
                     excel            0.000000          1.983294   
coag_and_floc        python           1.009428          0.098410   
                     excel            1.114695          0.108666   
tri_media_filtration python           0.000000          4.243153   

                             total_fixed_op_cost  
sw_onshore_intake    python             0.292658  
                     excel              0.302891  
coag_and_floc        python             0.164134  
                     excel              0.223931  
tri_media_filtration python             0.178538

In [18]:
df.loc['backwash_solids_handling']

,fixed_cap_inv,total_cap_investment,cat_and_chem_cost,electricity_cost,total_fixed_op_cost
python,0.079330,0.083416,0.0,0.000219,0.001111
excel,14.722875,15.481103,0.0,0.245464,0.212460


In [10]:
df = get_results_table()

In [11]:
df.to_csv("tmp/test_compare.csv")

In [15]:
m.fs.backwash_solids_handling.costing.electricity_cost()

0.00021929701170052268

In [18]:
cat_chem_df = pd.read_csv('data/catalyst_chemicals.csv', index_col = "Material")

In [13]:
import pandas as pd

In [20]:
df = pd.read_csv("data/case_study_results.csv")

In [16]:
wt.case_study_trains.train

{'case_study': 'Carlsbad', 'reference': 'NAWI', 'scenario': 'Baseline'}

In [17]:
train = wt.case_study_trains.train

In [22]:
df[df.case_study == train['case_study']]
df[df.scenario == train['scenario']]
df[df.unit_process == str(b_unit)[3:]]
print(df[df.Variable == "Fixed Capital Investment (FCI)"])

,Unnamed: 0,case_study,scenario,train,unit_process,Variable,unit,value,variable
0,0,Carlsbad,Baseline,A,sw_onshore_intake,Fixed Capital Investment (FCI),$MM,20.982722,fixed_cap_inv
1,1,Carlsbad,Baseline,A,sw_onshore_intake,Land_Cost,$MM,0.031474,land_cost
2,2,Carlsbad,Baseline,A,sw_onshore_intake,Working Capital,$MM,1.049136,working_cap
3,3,Carlsbad,Baseline,A,sw_onshore_intake,Total Capital Investment (TCI),$MM,22.063332,total_cap_investment
4,4,Carlsbad,Baseline,A,sw_onshore_intake,Catalysts and Chemicals,$MM/yr,0.000000,cat_and_chem_cost


In [23]:
print(b_unit)

fs.mixer2.mixed_state


In [ ]:
Excel_variable
Python_variable
Unit

Fixed Capital Investment (FCI)
fixed_cap_inv
$MM

Catalysts and Chemicals
cat_and_chem_cost
$MM/yr

Electricity
electricity_cost
$MM/yr

Other Operating
other_var_cost
$MM/yr

Total Fixed Operating 
total_fixed_op_cost
$MM/yr

Levelized Cost
LCOW
$/m3






In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)